In [ ]:
# uncomment if running in colab
# !pip install rdflib
# !pip install torch_geometric

In [ ]:
from rdflib import Graph, URIRef, BNode, Literal
from rdflib.namespace import FOAF, RDF
import networkx as nx
import numpy as np
from PIL import Image, ImageDraw
import random 
from collections import Counter
from itertools import combinations
import matplotlib.pyplot as plt
import statistics
import sys
import wave
import math
import struct
import argparse
from itertools import *
import noise1
import target1
import exp
import regex as re

In [ ]:
from base64 import urlsafe_b64encode
import gzip
from io import BytesIO
from rdf import Triple, URIRef, Literal
import datetime
from datetime import timedelta

In [ ]:
import nltk
import torch
from torch_geometric.data import Data
from collections import defaultdict, Counter
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import pandas as pd
from torch_geometric.utils import to_networkx

In [ ]:
# vocabulary retrieved from http://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain
_STRING_VOCAB = "./vocab.txt.gz"
_ATTR_AVAILABLE = ["xsd.string", "xsd.gYear", "xsd.integer", "xsd.float", 
                    "ogc.wktLiteral", "blob.image", "xsd.date",
                    "xsd.dateTime", "xsd.anyURI", "blob.audio"]
_ATTR_INCLUDED = [attr for attr in _ATTR_AVAILABLE]
_NAMESPACE = "https://example.org/"
_NAMESPACE_XSD = "http://www.w3.org/2001/XMLSchema#"
_NAMESPACE_OGC = "http://www.opengis.net/ont/geosparql#"
_NAMESPACE_KGB = "http://kgbench.info/dt#"

In [ ]:
def im_to_b64(im):
    buff = BytesIO()
    im.save(buff, format="JPEG")
    b64img = urlsafe_b64encode(buff.getvalue())

    return b64img.decode()

In [ ]:
def audio_to_b64_rand(nchannels, sampwidth, framerate, nframes):
    audio = noise1.gen_audio(44100,0.1,2, nsamples=None)
    w = wave.open('rand.wav','w')
    w.setparams((nchannels,sampwidth,framerate,nframes,'NONE', 'not compressed'))
    w.writeframesraw(audio)
    w.close()
    b64audio = urlsafe_b64encode(audio)
    
    return b64audio.decode()

In [ ]:
def audio_to_b64_targ(assignment, nchannels, sampwidth, framerate, nframes):
    audio = target1.gen_audio(assignment,44100,0.1)
    w = wave.open('targ.wav', 'w')
    #w.setparams((nchannels, sampwidth, framerate, nframes, 'NONE', 'not compressed'))
    w.setparams((2,2,44100,44100,'NONE', 'not compressed'))
    w.writeframesraw(audio)
    w.close()
    b64audio = urlsafe_b64encode(audio)

    return b64audio.decode()

In [ ]:
def gen_vocab():
    vocab = list()
    with gzip.open(_STRING_VOCAB, 'rt') as f:
        vocab = [word for word in f.read().split('\n') if len(word) >= 5]

    return vocab

In [ ]:
vocab = gen_vocab()

In [ ]:
def gen_random_attr(attr, vocab):
    if attr == "xsd.string":
        o = Literal(noise1.gen_string(vocab), dtype=_NAMESPACE_XSD+"string")
    elif attr == "xsd.anyURI":
        o = Literal(noise1.gen_anyURI(vocab), dtype=_NAMESPACE_XSD+"anyURI")
    elif attr == "xsd.gYear":
        o = Literal(str(noise1.gen_gYear()), dtype=_NAMESPACE_XSD+"gYear")
    elif attr == "xsd.date":
        o = Literal(str(noise1.gen_date()), dtype=_NAMESPACE_XSD+"date")
    elif attr == "xsd.dateTime":
        o = Literal(str(noise1.gen_dateTime()), dtype=_NAMESPACE_XSD+"dateTime")
    elif attr == "xsd.integer":
        o = Literal(str(noise1.gen_integer()), dtype=_NAMESPACE_XSD+"integer")
    elif attr == "xsd.float":
        o = Literal(str(noise1.gen_float()), dtype=_NAMESPACE_XSD+"float")
    elif attr == "xsd.boolean":
        o = Literal(str(noise1.gen_boolean()), dtype=_NAMESPACE_XSD+"boolean")
    elif attr == "blob.image":
        o = Literal(im_to_b64(noise1.gen_image()), dtype=_NAMESPACE_KGB+"base64Image")
    elif attr == "ogc.wktLiteral":
        o = Literal(noise1.gen_wktLiteral(), dtype=_NAMESPACE_OGC+"wktLiteral")
    elif attr == "blob.audio":
        o = Literal(audio_to_b64_rand(2,2,44100,44100), dtype=_NAMESPACE_XSD+"base64Audio")
    else:
        o = Literal("")  # empty literal

    return o

In [ ]:
def gen_target_attr(attr, vocab, assignment, nclusters):
    if attr == "xsd.string":
        o = Literal(target1.gen_string(vocab, assignment, nclusters), dtype=_NAMESPACE_XSD+"string")
    elif attr == "xsd.anyURI":
        o = Literal(target1.gen_anyURI(vocab, assignment, nclusters), dtype=_NAMESPACE_XSD+"anyURI")
    elif attr == "xsd.gYear":
        o = Literal(str(target1.gen_gYear(assignment, nclusters)), dtype=_NAMESPACE_XSD+"gYear")
    elif attr == "xsd.date":
        o = Literal(str(target1.gen_date(assignment, nclusters)), dtype=_NAMESPACE_XSD+"date")
    elif attr == "xsd.dateTime":
        o = Literal(str(target1.gen_dateTime(assignment, nclusters)), dtype=_NAMESPACE_XSD+"dateTime")
    elif attr == "xsd.integer":
        o = Literal(str(target1.gen_integer(assignment, nclusters)), dtype=_NAMESPACE_XSD+"integer")
    elif attr == "xsd.float":
        o = Literal(str(target1.gen_float(assignment, nclusters)), dtype=_NAMESPACE_XSD+"float")
    elif attr == "xsd.boolean":
        o = Literal(str(target1.gen_boolean(assignment)), dtype=_NAMESPACE_XSD+"boolean")
    elif attr == "blob.image":
        o = Literal(im_to_b64(target1.gen_image(assignment, nclusters)), dtype=_NAMESPACE_KGB+"base64Image")
    elif attr == "ogc.wktLiteral":
        o = Literal(target1.gen_wktLiteral(assignment, nclusters), dtype=_NAMESPACE_OGC+"wktLiteral")
    elif attr == "blob.audio":
        o = Literal(audio_to_b64_targ(assignment,2,2,44100,44100), dtype=_NAMESPACE_XSD+"base64Audio")
    else:
        o = Literal("")  # empty literal


    return o

In [ ]:
## Generate KG with desired parameter values, class distributions, attribute values, etc.

def generate(n_nodes, num_relations, num_targets,
             num_target_clusters,
             target_cluster_distribution, cluster_strength = 1.0,
             p_attr=0.9,density=None, symm=None, reflex=None,
             in_deg=None, in_ratio=None,
             out_deg=None, out_ratio=None,
             n_hops=None, hops_ratio=None,
             clustering=None, 
             diameter=None,
             avg_short_path=None, avg_long_path=None):
    assert len(target_cluster_distribution) == num_target_clusters

    g, edges = noise1.gen_structure(n_nodes, density=density, symm=symm, reflex=reflex,
             in_deg=in_deg, in_ratio=in_ratio,
             out_deg=out_deg, out_ratio=out_ratio,
             n_hops=n_hops, hops_ratio=hops_ratio,
             clustering=clustering, 
             diameter=diameter,
             avg_short_path=avg_short_path, avg_long_path=avg_long_path)
    
    context = set()
    if num_relations<3:
        for s, o in edges:
            p = np.random.choice(range(num_relations))
            t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                       URIRef(_NAMESPACE + "edge_" + str(p)),
                       URIRef(_NAMESPACE + "NODE_" + str(o)))

            context.add(t.ntriple().encode())
        
    else:
        for s, o in edges:
            if (o,s) in edges and s!=o:
                t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                       URIRef(_NAMESPACE + "edge_" + str(num_relations-1)),
                       URIRef(_NAMESPACE + "NODE_" + str(o)))
            elif s==o:
                t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                       URIRef(_NAMESPACE + "edge_" + str(num_relations-2)),
                       URIRef(_NAMESPACE + "NODE_" + str(o)))
            else:
                p = np.random.choice(range(num_relations-2))
                t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                           URIRef(_NAMESPACE + "edge_" + str(p)),
                           URIRef(_NAMESPACE + "NODE_" + str(o)))
            context.add(t.ntriple().encode())
    assert n_nodes >= num_targets
    instances = np.arange(n_nodes, dtype=np.int32)
    np.random.shuffle(instances)

    target_entities_idx = instances[:num_targets]
    non_target_entities_idx = instances[num_targets:]    
    attr_relations = dict()
    for attr in _ATTR_INCLUDED:
        attr_relations[attr] = np.random.choice(range(1,4))
    
    # add attributes to non-target entities
    for s in non_target_entities_idx:
        for attr in _ATTR_INCLUDED:
            if np.random.rand() > p_attr:
                continue
            attr_rel = np.random.choice(range(1, attr_relations[attr]+1))
            t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                       URIRef(_NAMESPACE + "edge_" + str(attr) + '_' + str(attr_rel)),
                       gen_random_attr(attr, vocab))

            context.add(t.ntriple().encode())
            
    # assign node labels starting from seeding process and add attributes to targets
    targets = [list() for _ in range(num_target_clusters)]
    counter = 0
    class_assign = dict()
    class_counts = dict()
    class_targ = dict()
    class_ratio = dict()
    n_num_target_clusters = num_target_clusters
    c = 0
    for i in target_cluster_distribution:
        class_targ[str(c)] = int(i * n_nodes)+1
        class_ratio[str(c)] = i
        class_counts[str(c)] = 0
        c+=1
    k=0
    for s in target_entities_idx:
        if counter<= num_target_clusters-1:
            assignment = k%(num_target_clusters)            
            class_assign[str(s)] = assignment
            class_counts[str(assignment)] = class_counts.get(str(assignment))+1
            nearby = [n for n in nx.generators.ego_graph(g, s, radius=4)]
            inc = [n[0] for n in g.in_edges(s)]
            nearby = nearby + inc
            nearby = [n for n in nearby if class_assign.get(str(n)) == None]
            if len(nearby) > 0:
                cluster_max = int((n_nodes * target_cluster_distribution[k])/10)
                if len(nearby) > cluster_max:
                    nearby = np.random.choice(nearby, size=cluster_max, replace=False)
                    for j in nearby:
                        if class_assign.get(str(j)) ==None:
                            if np.random.rand() < cluster_strength:
                                class_assign[str(j)] = assignment
                                class_counts[str(assignment)] = class_counts.get(str(assignment))+1
                else:
                    for j in nearby:
                        if class_assign.get(str(j)) ==None:
                            if np.random.rand() < cluster_strength:
                                class_assign[str(j)] = assignment
                                class_counts[str(assignment)] = class_counts.get(str(assignment))+1
            k += 1
            counter+=1
        else:
            if class_assign.get(str(s))!= None:
                assignment = class_assign[str(s)]
            else:
                exclude = []
                for i in range(len(class_targ)):
                    if class_targ.get(str(i)) < class_counts.get(str(i)) and len(n_target_cluster_distribution)>1:
                        exclude.append(i)
                for i in exclude:
                    class_ratio.pop(str(i), None)
                n_num_target_clusters = [int(n) for n in class_ratio.keys()]
                n_target_cluster_distribution = [n for n in class_ratio.values()]
                n_target_cluster_distribution = np.asarray(n_target_cluster_distribution).astype('float64')
                n_target_cluster_distribution = n_target_cluster_distribution / np.sum(n_target_cluster_distribution)
                neighbors = [n for n in g.neighbors(s)]
                if len(neighbors) > 0:
                    dig = []
                    for i in neighbors:
                        b = class_assign.get(str(i))
                        if b != None and (b not in exclude):
                            dig.append(b)
                    if len(dig) > 0:
                        r = statistics.mode(dig)
                        if np.random.rand() < cluster_strength:
                            assignment = np.int32(r)
                            class_assign[str(s)] = assignment
                            class_counts[str(assignment)] = class_counts.get(str(assignment))+1
                        else:
                            assignment = np.random.choice(n_num_target_clusters,
                                          p=n_target_cluster_distribution)
                            class_assign[str(s)] = assignment
                            class_counts[str(assignment)] = class_counts.get(str(assignment))+1
                    else:
                        assignment = np.random.choice(n_num_target_clusters,
                                      p=n_target_cluster_distribution)
                        class_assign[str(s)] = assignment
                        class_counts[str(assignment)] = class_counts.get(str(assignment))+1
                else:
                    assignment = np.random.choice(n_num_target_clusters,
                                  p=n_target_cluster_distribution)
                    class_assign[str(s)] = assignment
                    class_counts[str(assignment)] = class_counts.get(str(assignment))+1
                    
        for attr in _ATTR_INCLUDED:
            if np.random.rand() > p_attr:
                continue

            attr_rel = np.random.choice(range(1, attr_relations[attr]+1))

            t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                       URIRef(_NAMESPACE + "edge_" + str(attr) + '_' + str(attr_rel)),
                       gen_target_attr(attr, vocab,
                                       assignment, num_target_clusters))

            context.add(t.ntriple().encode())

        # add target signal
        t = Triple(URIRef(_NAMESPACE + "NODE_" + str(s)),
                   URIRef(_NAMESPACE + "hasClass"),
                   URIRef(_NAMESPACE + "NODE_CLASS_" + str(assignment)))

        targets[assignment].append(t.ntriple().encode())

    with gzip.open("./context.nt.gz", "wb") as gf:
        gf.write(b'\n'.join(context) + b'\n')   
    return(context, targets), g

In [ ]:
## run for normal synthetic KGs

# num_nodes = 100
# num_rel = 4
# num_targs = 100
# kg, G1 = generate(num_nodes,num_rel,num_targs,num_target_clusters=7, cluster_strength=1.0,
#               target_cluster_distribution=[.143,.143,.143,.143,.143,.143,.142],
#               density=0.00144, symm=None, reflex=None,
#              in_deg=None, in_ratio=None,
#              out_deg=None, out_ratio=None,
#              n_hops=None, hops_ratio=None,
#              clustering=None, 
#              diameter=None,
#              avg_short_path=None, avg_long_path=None)


In [ ]:
## run for Cora-syn

num_nodes = 2708
num_rel = 1
num_targs = 2708
kg, G1 = generate(num_nodes,num_rel,num_targs,num_target_clusters=7, cluster_strength=1.0,
              target_cluster_distribution=[.13,.08,.15,.30,.16,.11,.07],
              density=0.00144, symm=None, reflex=None,
             in_deg=None, in_ratio=0.01,
             out_deg=None, out_ratio=0.01,
             n_hops=None, hops_ratio=None,
             clustering= 0.240673298501937, 
             diameter=None,
             avg_short_path=None, avg_long_path=None)


In [ ]:
def plot_network(data):
    # visualize graph structure
    
    G = to_networkx(data, to_undirected=False)
    pos = nx.spring_layout(G, seed=42)
    cent = nx.degree_centrality(G)
    node_size = list(map(lambda x: x * 500, cent.values()))
    cent_array = np.array(list(cent.values()))
    threshold = sorted(cent_array, reverse=True)[10]
    cent_bin = np.where(cent_array >= threshold, 1, 0.1)
    plt.figure(figsize=(12, 12))
    nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
                                   cmap=plt.cm.plasma,
                                   node_color=cent_bin,
                                   nodelist=list(cent.keys()),
                                   alpha=cent_bin)
    edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3, arrowsize = 1)
    plt.show()

In [ ]:
def plot_classes(data):
    # visualize class clustering
                 
    G = to_networkx(data, to_undirected=False)

    label_dict = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 5,
        6: 6}
    node_color = []
    nodelist = [[], [], [], [], [], [], []]
    colorlist = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628']
    labels = data.y
    for n, i in enumerate(labels):
        node_color.append(colorlist[i])
        nodelist[i].append(n)
    pos = nx.spring_layout(G, seed = 42)
    plt.figure(figsize = (10, 10))
    labellist = list(label_dict.values())
    for num, i in enumerate(zip(nodelist, labellist)):
        n, l = i[0], i[1]
        nx.draw_networkx_nodes(G, pos, nodelist=n, node_size = 5, node_color = colorlist[num], label=l)
    nx.draw_networkx_edges(G, pos, width = 0.25, arrowsize = 1)
    plt.legend(bbox_to_anchor=(1, 1), loc='upper left')

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, data.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
results = []
for i in range(10):
    train,test,valid = exp.split(kg)
    with gzip.open("./train.nt.gz", "wb") as gf:
        gf.write(b'\n'.join(train) + b'\n')
    with gzip.open("./test.nt.gz", "wb") as gf:
        gf.write(b'\n'.join(test) + b'\n')
    with gzip.open("./valid.nt.gz", "wb") as gf:
        gf.write(b'\n'.join(valid) + b'\n')
        
    train, test, valid = exp.node_ind(train,test,valid)
    labs,nodes = exp.labels(kg)
    tr_mask, tes_mask, val_mask = exp.masks(nodes,train,test,valid)
    edge_index, edge_feats = exp.edges_info(kg,nodes)
    x = torch.tensor(np.identity(len(nodes)),dtype=torch.float)
    data = Data(x = x, edge_index=edge_index, 
              y =labs, train_mask= tr_mask, test_mask=tes_mask, val_mask = val_mask, 
                num_classes=7, num_nodes=num_nodes)
    
    # show visuals for graph
    if i == 0:
        exp.class_dist(labs, tr_mask, tes_mask, val_mask)
        plot_network(data)
        plot_classes(data)
        counter = Counter(data.y.numpy())
        counter = dict(counter)
        count = [x[1] for x in sorted(counter.items())]
        plt.figure(figsize=(10, 6))
        plt.bar(range(7), count)
        plt.title("Class distribution of node labels")
        plt.xlabel("Class")
        plt.ylabel("Total nodes")
        plt.show()
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = GCN()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()
    val_accs = []
    for epoch in range(200):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        model.eval()
        pred = model(data).argmax(dim=1)
        correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
        accuracy = int(correct) / int(data.test_mask.sum())
        val_accs.append(accuracy)
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    results.append(acc)
    print(f'Accuracy: {acc:.4f}')
    plt.figure(figsize=(12,8))
    plt.plot(np.arange(1, len(val_accs) + 1), val_accs, label='Validation accuracy', c='blue')
    plt.xlabel('Epochs')
    plt.ylabel('Accurarcy')
    plt.title('GCN validation performance')
    plt.legend(loc='upper right', fontsize='x-large')
    plt.show()

In [ ]:
# Cora dataset

from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]
x = torch.tensor(np.identity(2708),dtype=torch.float)

In [ ]:
print(data)

In [ ]:
data = Data(x = x, edge_index=dataset[0].edge_index, y =dataset[0].y, train_mask= dataset[0].train_mask, test_mask=dataset[0].test_mask, val_mask = dataset[0].val_mask, num_classes=7)

In [ ]:
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(data.num_node_features, 16)
        self.conv2 = GCNConv(16, 7)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

rs = []
for i in range(10):
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.test_mask] == data.y[data.test_mask]).sum()
    acc = int(correct) / int(data.test_mask.sum())
    rs.append(acc)
    print(f'Accuracy: {acc:.4f}') 

In [ ]:
## Plot Cora network
## code from: https://medium.com/mlearning-ai/ultimate-guide-to-graph-neural-networks-1-cora-dataset-37338c04fe6f

G = to_networkx(data, to_undirected=True)
pos = nx.spring_layout(G, seed=42)
cent = nx.degree_centrality(G)
node_size = list(map(lambda x: x * 500, cent.values()))
cent_array = np.array(list(cent.values()))
threshold = sorted(cent_array, reverse=True)[10]
print("threshold", threshold)
cent_bin = np.where(cent_array >= threshold, 1, 0.1)
plt.figure(figsize=(12, 12))
nodes = nx.draw_networkx_nodes(G, pos, node_size=node_size,
                               cmap=plt.cm.plasma,
                               node_color=cent_bin,
                               nodelist=list(cent.keys()),
                               alpha=cent_bin)
edges = nx.draw_networkx_edges(G, pos, width=0.25, alpha=0.3)
plt.show()

In [ ]:
## Plot Cora classes
## code from: https://medium.com/mlearning-ai/ultimate-guide-to-graph-neural-networks-1-cora-dataset-37338c04fe6f

label_dict = {
        0: 0,
        1: 1,
        2: 2,
        3: 3,
        4: 4,
        5: 5,
        6: 6}
G = to_networkx(data, to_undirected=True)
node_color = []
nodelist = [[], [], [], [], [], [], []]
colorlist = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628']
labels = data.y
for n, i in enumerate(labels):
    node_color.append(colorlist[i])
    nodelist[i].append(n)
pos = nx.spring_layout(G, seed = 42)
plt.figure(figsize = (10, 10))
labellist = list(label_dict.values())
for num, i in enumerate(zip(nodelist, labellist)):
    n, l = i[0], i[1]
    nx.draw_networkx_nodes(G, pos, nodelist=n, node_size = 5, node_color = colorlist[num], label=l)
nx.draw_networkx_edges(G, pos, width = 0.25)
plt.legend(bbox_to_anchor=(1, 1), loc='upper left')

In [ ]:
## Visualize results from experiments

large = np.array([0.647,0.6514,0.6464,0.6534,0.6572,0.652,0.6674,0.6558,0.6604,0.6634])
medium = np.array([0.5803,0.5808,0.5963,0.5753,0.5669,0.5878,0.5778,0.5898,0.5753,0.5763])
small = np.array([0.3892,0.4236,0.3645,0.3695,0.4384,0.4039,0.3842,0.4039,0.399,0.3744])
cora_syn = np.array([0.5699,0.5956,0.5882,0.5827,0.5809,0.5993,0.5827,0.5901,0.5864,0.6103])
cora = np.array([0.622,0.612,0.612,0.615,0.639,0.642,0.593,0.638,0.601,0.614])
rs = np.array([0.8010, 0.8010, 0.8180, 0.8070, 0.8060, 0.8080, 0.8020, 0.8080, 0.8120, 0.7980])


large_mean = np.mean(large)
medium_mean = np.mean(medium)
small_mean = np.mean(small)
syn_mean = np.mean(cora_syn)
cora_mean = np.mean(cora)
rs_mean = np.mean(rs)

large_std = np.std(large)
medium_std = np.std(medium)
small_std = np.std(small)
syn_std = np.std(cora_syn)
cora_std = np.std(cora)
rs_std = np.std(rs)

graphs = ['KG Small', 'KG Medium', 'KG Small', 'Cora-syn', 'Cora']
x_pos = np.arange(len(graphs))
CTEs = [small_mean, medium_mean, large_mean, syn_mean, cora_mean]
error = [small_std, medium_std, large_std, syn_std, cora_std]

fig, ax = plt.subplots()
ax.bar(x_pos, CTEs, yerr=error, align='center', alpha=0.5, ecolor='black', capsize=10)
ax.set_ylabel('Average accuracy')
ax.set_xlabel('Graph type')
ax.set_xticks(x_pos)
ax.set_xticklabels(graphs)
ax.set_title('Perfomance results')
ax.yaxis.grid(True)
plt.tight_layout()
plt.show()